In [3]:
import sys
sys.path.insert(0, "../../")
from pyecharts import online
online()
#  演示必要的准备代码，使用该库时不需重复此单元格命令

In [1]:
import xalpha as xa
import pandas as pd

# 指数基金增强效果分析

## 按净值分析

In [2]:
# 选取几个典型的500增强进行分析
jianxin = xa.fundinfo('000478')
chuangjinhexin = xa.fundinfo('002311')
nanfang = xa.fundinfo('002906')

In [3]:
# 再选取中证500指数本身和普通500指数基金做比较
zz500 = xa.indexinfo('0000905')
guangfa = xa.fundinfo('162711')

In [4]:
analysis = xa.evaluate(jianxin,chuangjinhexin,nanfang,guangfa,zz500,start='2017-01-01')
# 导入 evaluate 类统一比较

In [5]:
analysis.v_netvalue()
# 同一个起点不同的轨迹，但还好都稳稳跑赢指数本身，说明增强确实有效果

In [6]:
analysis.correlation_table() # 看一看各家之间的关联系数，在保持了对500的高相关的情形下，实现了超额收益

,000478,002311,002906,162711,0000905
000478,1.000000,0.990764,0.981019,0.993353,0.992961
002311,0.990764,1.000000,0.979623,0.990808,0.990908
002906,0.981019,0.979623,1.000000,0.979778,0.979301
162711,0.993353,0.990808,0.979778,1.000000,0.999573
0000905,0.992961,0.990908,0.979301,0.999573,1.000000


In [7]:
# 以表现最好的一只单独研究
chuangjinhexin.bcmkset(zz500,start='2017-01-01')

In [8]:
chuangjinhexin.alpha(), chuangjinhexin.information_ratio() # 超额收益指标显著

(0.07608341594961437, 2.4104742808274997)

## 按持有分析
有些基金会大量分红，如果只按照净值来分析，就无法客观的描述其关联系数和超额收益，按总值分析，又忽略了分红再投入的收益，因此我们直接持有，按照分红再投入的策略模版交易，并形成可以净值刻画的封闭组合类来分析。

In [9]:
# 这次以喜欢分红的跟踪中证红利的基金为例
fuguo = xa.fundinfo('100032')
dacheng = xa.fundinfo('090010')
zzhl = xa.indexinfo('1399922')

In [10]:
fuguo.special # 富国多次分红，因此我们需要额外构造封闭组合类来模拟其真实的收益情况

,date,netvalue,comment,totvalue
243,2010-01-14,1.410,0.245,1.863
488,2011-01-19,1.151,0.130,1.718
729,2012-01-16,0.969,0.026,1.542
1458,2015-01-20,1.385,0.056,2.073
1712,2016-02-02,1.118,0.350,2.167
1945,2017-01-17,1.087,0.300,2.469
2191,2018-01-18,1.215,0.188,2.825
2701,2020-02-28,0.984,0.110,2.689


In [11]:
dacheng.special #大成这只没有分过红 

,date,netvalue,comment,totvalue
2233,2019-06-19,1.63,0.156,1.786


In [12]:
st = xa.policy.buyandhold(fuguo,'2011-01-01') # 制定一个一次性买入并一直持有，分红始终选择再投入的账单

In [13]:
fuguot = xa.trade(fuguo, st.status) # 按账单买入富国红利

In [14]:
fuguotclose = xa.mulfix(fuguot) #为了可以净值刻画，将这笔交易嵌入总值和上笔相同的封闭投资系统中

In [15]:
fuguotclose.bcmkset(zzhl, start='2011-01-01')  # 通过设定基准，来生成该投资系统的净值

In [16]:
fuguotclose.name = '富国中证红利增强'
fuguotclose.code = '100032'
# 为了使该系统被 evaluate 类接受，还需要手动添加名字和代码属性

In [17]:
analysis_zzhl = xa.evaluate(fuguotclose,dacheng,zzhl,start='2011-01-05')

In [18]:
analysis_zzhl.v_netvalue() # 富国很强，大家还是都能跑赢基准，
# 这种高分红的指数跑赢基准很容易，因为普通指数不是全收益指数，没有计入分红部分，而指数基金投资是能收到分红的

In [19]:
analysis_zzhl.correlation_table() # 关联性自然是杠杠的

,100032,090010,1399922
100032,1.000000,0.992491,0.995144
090010,0.992491,1.000000,0.996877
1399922,0.995144,0.996877,1.000000


In [20]:
# 你可能会问，如果直接有净值比会咋样，我们看一下可不可行
analysis_wrong = xa.evaluate(fuguo,dacheng,zzhl,start='2011-01-01')

In [21]:
analysis_wrong.v_netvalue() # 完全没办法比，富国的几次分红使得净值很低，无法看出真实的投资收益

In [22]:
analysis_wrong.correlation_table() # 用净值比较，富国的关联性出现严重失真

,100032,090010,1399922
100032,1.000000,0.835243,0.836889
090010,0.835243,1.000000,0.996877
1399922,0.836889,0.996877,1.000000


对于出现过基金份额折算的基金，常见于各种分级基金母基，也需要按照上面的按持有分析的思路，构造净值系统进行比较，这里不再赘述